### This is a simple implementation of Google Drive crawler 
#### - First we need to get authorization from Drive API  see https://developers.google.com/drive/api/v3/quickstart/python 
#### - Required credentials that should be in credential.json file
#### - Creat a folder in Drive "data" that contains different types of files txt, pdf, ppt ...

In [2]:
from __future__ import print_function
import pickle
import os.path
from oauth2client import file 
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import json
from oauth2client.file import Storage
SCOPES = ['https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/drive.file']

def authentificate():
    """Shows basic usage of the Drive v3 API.
    Prints the names and ids of the first 10 files the user has access to.
    """
    creds= None
    if os.path.exists('credentials.json'):
        with open('credentials.json') as token:
          store = Storage('credentials.json')
          creds = None
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('credentials.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('drive', 'v3', credentials=creds)
    return service

auth = authentificate()



Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=631444652840-gsquoh612j140throsdclukc4j0jnqgh.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A60902%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.file&state=eYKcECMDz7OkALnDLXzySjlRZ6UH6n&access_type=offline


In [3]:
def gdrive_get_all_files_in_folder(folder_name, auth):
    service = auth
    resu = []
    # Retrieving all files from a given folder 
    folder_root = service.files().list(q="mimeType ='application/vnd.google-apps.folder' and name= '{}'".format(folder_name) ,
         fields="nextPageToken, files(id, name)").execute()
    items = folder_root.get('files', [])
        
    if items :
        if(items[0] is not None):
            item = items[0]
#             print(u'{0} ({1})'.format(item['name'], item['id']))
            que =  service.files().list(q="'"+ item['id']+ "' in parents" ,
            fields="nextPageToken, files(id, name)").execute()
            if(que.get('files',[]) is not None):
                for k in que.get('files',[]):
                    x = gdrive_get_all_files_in_folder(k['name'], service)
                    if x is not None: resu.extend(x)
                    if x is None : resu.append((k['name'], k['id']))
                return resu
   
    

folder_of_interest = 'data'  
files = gdrive_get_all_files_in_folder(folder_of_interest, auth)

In [4]:
import io
from googleapiclient.http import MediaIoBaseDownload

def gdrive_download_file(file, path_to_save, service): 
    #Download file and save it under the path
    file_id = file[1]
    request = service.files().get_media(fileId=file_id)
    fh = io.BytesIO()
    if not os.path.exists(path_to_save):
        os.makedirs(path_to_save)
    downloader = MediaIoBaseDownload(fh, request)
    print(type(downloader))
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        with io.open(path_to_save+"/"+file[0],'wb') as fi:  #save with the name of the file
            fh.seek(0)
            fi.write(fh.read())
    print ("Download %d%%." % int(status.progress() * 100))
    

In [5]:
test_dir = r'/home/farah/test'
for item in files:
    gdrive_download_file(item, test_dir, auth)

<class 'googleapiclient.http.MediaIoBaseDownload'>
Download 100%.
<class 'googleapiclient.http.MediaIoBaseDownload'>
Download 100%.
<class 'googleapiclient.http.MediaIoBaseDownload'>
Download 100%.
<class 'googleapiclient.http.MediaIoBaseDownload'>
Download 100%.
<class 'googleapiclient.http.MediaIoBaseDownload'>
Download 100%.
<class 'googleapiclient.http.MediaIoBaseDownload'>
Download 100%.
<class 'googleapiclient.http.MediaIoBaseDownload'>
Download 100%.
<class 'googleapiclient.http.MediaIoBaseDownload'>
Download 100%.
<class 'googleapiclient.http.MediaIoBaseDownload'>
Download 100%.
<class 'googleapiclient.http.MediaIoBaseDownload'>
Download 100%.
<class 'googleapiclient.http.MediaIoBaseDownload'>
Download 100%.
<class 'googleapiclient.http.MediaIoBaseDownload'>
Download 100%.
<class 'googleapiclient.http.MediaIoBaseDownload'>
Download 100%.
<class 'googleapiclient.http.MediaIoBaseDownload'>
Download 100%.
<class 'googleapiclient.http.MediaIoBaseDownload'>
Download 100%.
<class 'go

In [14]:
import textract 
import re

import os, subprocess
from tika import parser


def get_file_strings(path):
    exten = {'.csv','.doc','.docx','.eml','.epub','.gif','.jpg','.json','.html','.msg','.odt','.ogg','.pdf','.png','.pptx','.ps','.rtf','.tiff','.txt','.wav','.xlsx','.xls'}
    extension = os.path.splitext(path)[1]
    if extension == '.pdf' :
        raw = parser.from_file(path)
        raw = str(raw).replace('\\n', '\n').replace('\\r', '').split('\n')
        return raw
    if extension=='.txt':
        with open(path, 'rb') as file:
            data = file.read().decode(errors='replace').replace('\\n', '\n').replace('\\r', '').split('\n')
            return data
    else: 
        if extension in exten:
            texts = str(textract.process(path)).replace('\\n', '\n').replace('\\r', '').split('\n')
            return texts
        if extension not in exten and extension != '.mp3' and extension !='.avi':
            with open(path, 'rb') as file:
                data = file.read().decode(errors='replace').replace('\\n', '\n').replace('\\r', '').split('\n')
                return data
                
# creating dictionary of parsed files
files_data = dict()
for file in os.scandir(test_dir):   
    print(file.name)
    strings = get_file_strings(file.path)
    if strings:
        files_data[file.name] = strings
        



3cases.pdf
ai-junior.pdf
Assessment Criteria (May).pdf
at least this file.txt
AY16-17 Academic Calendar .pdf
bloomset.js
cs.pdf
cyclomat.c
deep-features-scene (1).pdf
dsa.pdf
DSA_09 - 2-3-4 and B-Trees.pdf
DSA_15 Lion in the desert.pptx
FuncnNEW.pdf
grant-translate.txt
grant.txt
hockey.avi
L5-problems-2015.pdf
lockexamples.c
MS2 - Problems of multithread programming.pptx
neuro.html
nn.cpp
origin-05.mp3
origin-06.mp3
Program.cs
rdtsc-gcc.c
rdtsc-vc.cpp
retake-2016-08-18.docx
skiplist.js
Small dataset face recognition.pptx
sort.js
students.txt
Tutorial #8.pdf
Tutorial 9.pdf
[DM]-Course Description.docx
